In [4]:
# !pip install --upgrade transformers
import torch
import sys
sys.path.append(r'../../')
# import Merak

In [5]:
linear = torch.nn.Linear(1,1)

In [3]:
from transformers import BertForMaskedLM, BertConfig
model = BertForMaskedLM(BertConfig(num_hidden_layers=1))
print(model)

from transformers.utils.fx import symbolic_trace as trace1
# from Merak.autoshard.convert import symbolic_trace as trace2
from torch.fx import symbolic_trace as trace3 # cannot be used for transformer models.



BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [4]:
trace1(model) # compared to trace2, it cannot capture LinearProxy

{<built-in method full of type object at 0x1036565d0>, <built-in method full_like of type object at 0x1036565d0>, <built-in method eye of type object at 0x1036565d0>, <built-in method zeros of type object at 0x1036565d0>, <built-in method ones of type object at 0x1036565d0>, <built-in method empty of type object at 0x1036565d0>, <built-in method arange of type object at 0x1036565d0>, <built-in method tensor of type object at 0x1036565d0>}
BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=7

/Users/connolly/opt/anaconda3/envs/python3/lib/python3.7/site-packages/torch/fx/_symbolic_trace.py:420: UserWarning: Was not able to add assertion to guarantee correct inputs to specialized function. It is up to the user to make sure that your inputs match the inputs you specialized the function with.
  "Was not able to add assertion to guarantee correct inputs to "


GraphModule(
  (bert): Module(
    (embeddings): Module(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (token_type_embeddings): Embedding(2, 768)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): Module(
      (layer): Module(
        (0): Module(
          (attention): Module(
            (self): Module(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): Module(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=

In [10]:
trace2(model)

NameError: name 'trace2' is not defined

In [14]:
trace3(model.bert)

ValueError: You cannot specify both input_ids and inputs_embeds at the same time

In [21]:
class LinearProxy(torch.nn.Module):
    def __init__(self, in_features, out_features, bias=True, device=None, dtype=None):
        self.module_args = (in_features, out_features, bias, device, dtype)
        super(LinearProxy, self).__init__()
        global SHOULD_PRINT_LINEAR
        if SHOULD_PRINT_LINEAR:
            print_rank_0('using linear proxy')
            SHOULD_PRINT_LINEAR = False
        self.in_features = in_features
        self.out_features = out_features
        self._bias = bias

        self.mp_attr = ' '

        w = torch.empty(1, 1)
#         self.weight = NumelParameter(w)
        self.weight.num_element = lambda: self.out_features*self.in_features

        if bias:
#             self.bias = NumelParameter(torch.zeros(1))
            self.bias.num_element = lambda: self.out_features
        else:
            self.register_parameter('bias', None)


    def build(self, init_args):
        if self.mp_attr == ' ':
            return torch.nn.Linear(*self.module_args).cuda()
        if self.mp_attr.startswith('row'):
            init_method = scaled_init_method_normal(*init_args)
            if self.mp_attr == 'row_mlp':
                return RowPara(self.module_args[0], self.module_args[1], init_method, bias=self._bias)
            else:
                return RowPara(self.module_args[0], self.module_args[1], init_method, bias=self._bias, need_permute=False)
        elif self.mp_attr.startswith('col'):
            init_method = init_method_normal(init_args[0])
            if self.mp_attr == 'col_mlp':
                return ColPara(self.module_args[0], self.module_args[1], init_method, bias=self._bias)
            else:
                return ColPara(self.module_args[0], self.module_args[1], init_method, bias=self._bias, need_permute=False)
        
        raise NotImplementedError(f"Not supported model/tensor parallelism layers {self.mp_attr}")
        

    def extra_repr(self) -> str:
        return 'in_features={}, out_features={}, bias={}'.format(
            self.in_features, self.out_features, self.bias is not None
        )
    
    def forward(self, x):
        # shape = x.shape
        # shape[-1] = self.out_features
        # if len(x.shape) == 2:
        #     return x[:, :1].expand(-1, self.out_features).contiguous()
        try:
            return x[:, :, :1].expand(-1,-1, self.out_features).contiguous()#, device=x.device)
        except IndexError:
            return x[:, :1].expand(-1, self.out_features).contiguous()

In [10]:
# with 语句替换linear proxy?
class Replacer:
    def __enter__(self):
        print("Replacing torch.nn.Linear with Proxy linear.")
        torch.nn.Linear = LinearProxy
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        import importlib
        importlib.reload(torch.nn)
        print("Reloading torch.nn")
        
# with Replacer() as replace:
#     linear = torch.nn.Linear(10,10)
#     print(linear)

# print(linear)

# linear = torch.nn.Linear(10,10)
# print(linear)
    
print('-'*50)
    
def proxy_linear(func):
    def wrapper(*args, **kwargs):
        with Replacer() as replace:
            return func(*args, **kwargs)
    return wrapper


class toyModule(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.dense1 = torch.nn.Linear(10,10)
        self.dense2 = torch.nn.Linear(10,1)
    
    def forward(self, inputs):
        out = self.dense1(inputs)
        out = self.dense2(out)
        return out

class toyModule2(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.toy = toyModule()
    
    def forward(self, inputs):
        return self.toy(inputs)

class MerakTrainer():
    @proxy_linear
    def __init__(self, module):
        super().__init__()
        self.model = module()
    def __repr__(self):
        return str(self.model)

# model = MerakTrainer(toyModule2)
# print(model)

model = toyModule2()

inputs = torch.rand(1,10)
graph = trace3(model).graph
for node in graph.nodes:
    print(node)

--------------------------------------------------
inputs
toy_dense1
toy_dense2
output


In [17]:
model.toy.__module__

'__main__'

In [3]:
trace3(model)

graph():
fn: <function BertForMaskedLM.forward at 0x7ff01d3a8cb0>
args:[BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
     

ValueError: You cannot specify both input_ids and inputs_embeds at the same time

In [1]:
import torch
from torch.fx import symbolic_trace, replace_pattern


'''
How to Use the FX Subgraph Rewriter
For easy subgraph rewriting, FX exposes the utility function:
    replace_pattern(gm : GraphModule,
                    pattern : Callable,
                    replacement : Callable)
                    -> None
`replace_pattern` matches all possible non-overlapping sets of operators
and their data dependencies (`pattern`) in the Graph of a GraphModule
(`gm`), then replaces each of these matched subgraphs with another
subgraph (`replacement).
The docstring for `replace_pattern` (located in `subgraph_rewriter.py`)
gives an in-depth explanation as to how `pattern` and `replacement`
should be specified, what happens during pattern matching, and other
important technical details. This tutorial, therefore, is only meant to
give an overview as to the FX Subgraph Rewriter's basic functionality.
Let's go rewrite a Graph!
'''

# Sample module
class M(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x, w1, w2):
        val1 = torch.neg(w1)
        m1 = torch.cat([val1, w2]).sum()
        val2 = torch.neg(w1)
        m2 = torch.cat([val2, w2]).sum()
        return x + torch.max(m1) + torch.max(m2)

# Symbolically trace an instance of `M`
traced = symbolic_trace(M())

# Define the pattern. The FX Subgraph Rewriter will match all
# non-overlapping instances of the pattern in the larger graph.
# Note that Pattern-matching is done based on data dependencies,
# not Node names. Even though we're operating on Nodes named `a1` and
# `a2` instead of `w1` and `w2`, the pattern is still a valid match
# for the two instances of `torch.cat([w1, w2]).sum()` above. Only
# operations that contribute to the single output value of the pattern
# are considered
def pattern(a1, a2):
    val1 = torch.neg(a1)
    return torch.cat([val1, a2]).sum()

# Define the replacement (same rules as the pattern)
def replacement(w1, w2):
    return torch.stack([w1, w2])

# Replace `pattern` with `replacement` in `traced`
print(replace_pattern(traced, pattern, replacement))
print(traced)



 x
False output placeholder output x


 w1
False output placeholder output w1


 w2
False output placeholder output w2


 neg
True output call_function output <built-in method neg of type object at 0x10e90b5d0>
False call_method placeholder sum w1


 cat
True output call_function output <built-in method cat of type object at 0x10e90b5d0>
False call_method call_function sum <built-in method neg of type object at 0x10e90b5d0>
False call_method placeholder sum w2


 sum_1
True output call_method output sum
False call_method call_function sum <built-in method cat of type object at 0x10e90b5d0>


 neg_1
True output call_function output <built-in method neg of type object at 0x10e90b5d0>
False call_method placeholder sum w1


 cat_1
True output call_function output <built-in method cat of type object at 0x10e90b5d0>
False call_method call_function sum <built-in method neg of type object at 0x10e90b5d0>
False call_method placeholder sum w2


 sum_2
True output call_method output sum
False c

In [3]:
from torch.fx.subgraph_rewriter import _SubgraphMatcher

matcher = _SubgraphMatcher(symbolic_trace(pattern).graph)

In [10]:
print(matcher.pattern_anchor.target)

output
